Chapter 16 Trees<br>

- A Tree is a data type that is ideal for representing hierarchical structure.
- Trees are composed of nodes and nodes have 0 or more children or child nodes. 
- A node is called the parent of its children.
- Each node has (at most) one parent.
- If the children are ordered in some way, then we have an ordered tree.
- There is a single special node called the root of the tree.
- The root is the only node that does not have a parent.
- The nodes that do not have any children are called leaves or leaf nodes.

16.1 Some more definitions<br>

- A path in a tree is a sequence of nodes in which each node is a child of the previous node. 
- We say it is a path from x to y if the first node is x and the last node is y.
- There exists a path from the root to every node in the tree.
- The length of the path is the number of hops or edges which is one less than the number of nodes in the path.
- The descendants of a node x are all those nodes y for which there is a path from x to y. If y is a descendant of x, then we say x is an ancestor of y.
- Given a tree T and a node n in that tree, the subtree rooted at n is the tree whose root is n that contains all descendants of n.
- The depth of a node is the length of the path to the node from the root.
- The height of a tree is the maximum depth of any node in the tree.

16.2 A recursive view of trees<br>

We can use lists to represent a hierarchical structure by making lists of lists.<br>
Example:<br>
```['a', ['p'], ['n'], ['t']]```<br>

![Alt text](figs/tree_example1.png)<br>

This is a tree with 'a' stored in the root. The root has 3 children
storing respectively, 'p', 'n', and 't'.<br>

```T = ['c', ['a', ['p'], ['n'], ['t']], ['o', ['n']]]```<br>

![Alt text](figs/tree_example2.png)

In [1]:
def printtree(T):
    print(T[0])
    for child in range(1, len(T)):
        printtree(T[child])

T = ['c', ['a', ['p'], ['n'], ['t']], ['o', ['n']]]
printtree(T)

c
a
p
n
t
o
n


In [2]:
def printtree(T):
    iterator = iter(T)
    print(next(iterator))
    for child in iterator:
        printtree(child)

printtree(T)

c
a
p
n
t
o
n


16.3 A Tree ADT<br>

The information in the tree is all present in the list of lists structure. How-
ever, it can be cumbersome to read, write, and work with. We will package
it into a class that allows us to write code that is as close as possible to how
we think about and talk about trees. As always, we will start with an ADT
that describes our expectations for the data structure and its usage.<br>
The Tree ADT is as follows.<br>
- __ init __ (L) : Initialize a new tree given a list of lists. The convention
is that the first element in the list is the data and the later elements
(if they exist) are the children.<br>
- height() : Return the height of the tree.
- __ str __ () : Return a string representing the entire tree.
- __ eq __ (other) : Return True if the tree is equal to other. This means that they have the same data and their children are equal (and in the same order).
- __ contains __(k) : Return True if and only if the tree contains the data k either at the root or at one of its descendants. Return False otherwise.
- preorder() Return an iterator over the data in the tree that yields values according to the preorder traversal of the tree.
- postorder() : Return an iterator over the data in the tree that yields values according to the postorder traversal of the tree.
- __ iter __ () : An alias for preorder.
- layerorder() : Return an iterator over the data in the tree that yields values according to the layer order traversal of the tree.

16.4 An implementation

In [3]:
class Tree:
    def __init__(self, L):
        iterator = iter(L)
        self.data = next(iterator)
        self.children = [Tree(c) for c in iterator]

    def __str__(self, level = 0):
        treestring = " " * level + str(self.data)
        for child in self.children:
            treestring += "\n" + child.__str__(level + 1)
        return treestring

The initializer takes a list of lists representation of a tree as input. A
Tree object has two attributes, data stores data associated with a node and
children stores a list of Tree objects. The recursive aspect of this tree is
clear from the way the children are generated as Tree’s. This definition does
not allow for an empty tree

In [4]:
def printtree(T):
    print(T.data)
    for child in T.children:
        printtree(child)

T = Tree(['a', ['b', ['c', ['d']]],['e',['f'], ['g']]])
printtree(T)

a
b
c
d
e
f
g


In [5]:
print(T)

a
 b
  c
   d
 e
  f
  g


Although the code above seems to work, it does something terrible. It
builds up a string by iterative adding more strings (i.e. with concatenation).
This copies and recopies some part of the string for every node in the tree.
Instead, we would prefer to just keep a nice list of the trees and then join
them into a string in one final act before returning. To do this, it is handy
to use a helper method that takes the level and the current list of trees as
parameters. With each recursive call, we add one to the level, and we pass
down the same list to be appended to.

In [6]:
class Tree:
    def __init__(self, L):
        iterator = iter(L)
        self.data = next(iterator)
        self.children = [Tree(c) for c in iterator]

    def _listwithlevels(self, level, trees):
        trees.append(" " * level + str(self.data))
        for child in self.children:
            child._listwithlevels(level + 1, trees)
    
    def __str__(self):
        trees = []
        self._listwithlevels(0, trees)
        return "\n".join(trees)
    
    def __eq__(self, other):
        return self.data == other.data and self.children == other.children
    
    def height(self):
        if len(self.children) == 0:
            return 0
        else:
            return 1 + max(child.height() for child in self.children)

    def __contains__(self, k):
        return self.data == k or any(k in ch for ch in self.children)    

    # classic example of postorder traversal

    def printpostorder(self):
        for child in self.children:
            self.printpostorder(child)
        print(T.data)

    def _preorder(self):
        yield self
        for child in self.children:
            for descendant in child._preorder():
                yield descendant

T = Tree(['a', ['b', ['c', ['d']]],['e',['f'], ['g']]])
print(str(T))

a
 b
  c
   d
 e
  f
  g


The pattern involved here is called a tree traversal and we will discuss these in more depth.<br>

Check if two trees are equal in the sense of having the same shape and data.<br>
We use the __ eq __ method so this method will be used when we use == to check equality between Tree’s.

A function that computes the height
of the tree. We can do this by computing the height of the subtrees and
return one more than the max of those. If there are no children, the height
is 0.

The any function takes an iterable of booleans and return True if any of
them are True. It handles short-circuited evaluation, so it can stop as soon
as it finds that one is true. If the answer is False, then this will iterate over
the whole tree.

16.5 Tree Traversal<br>

For trees, the process of visiting all the nodes is called tree traversal.<br>
For ordered trees, there are two standard traversals, called:
- preorder -> In a preorder traversal, we visit the node first followed by the traversal of its children.
- postorder -> In a postorder traversal, we traverse all the children and then visit the node itself. 

The *printtree* method given previously is a classic example of a preorder traversal and also the *printpostorder* method.

16.6 If you want to get fancy...

In [7]:
# just another way of preorder traversal
def preorder(self):
    yield self.data
    for child in self.children:
        for data in child.preorder():
            yield data

16.6.1 There’s a catch!<br>

the total running time is proportional to the sum of the depths of
all the nodes in the tree. For a degenerate tree (i.e. a single path), this is
***O(n2)*** time. For a perfectly balanced binary tree, this is ***O(n log n)*** time.<br>

Using recursion and the call stack make the tree traversal code substan-
tially simpler than if we had to keep track of everything manually. It would
not be enough to store just the stack of nodes in the path from your current
node up to the root. You would also have to keep track of your place in the
iteration of the children of each of those nodes. Remember that it is the job
of an iterator object to keep track of where it is in the iteration. Thus, we
can just push the iterators for the children onto the stack too.

In [8]:
def _postorder(self):
    node, childiter = self, iter(self.children)
    stack = [(node, childiter)]
    while stack:
        node, childiter = stack[-1]
        try:
            child = next(childiter)
            stack.append((child, iter(child.children)))
        except StopIteration:
            yield node
            stack.pop()

def postorder(self):
    return (node.data for node in self._postorder())

16.6.2 Layer by Layer<br>

non-recursive postorder traversal, we can modify it to
traverse the tree layer by layer.

In [9]:
def _layerorder(self):
    node, childiter = self, iter(self.children)
    queue = Queue()
    queue.enqueue((node, childiter))
    while queue:
        node, childiter = queue.peek()
    try:
        child = next(childiter)
        queue.enqueue((child, iter(child.children)))
    except StopIteration:
        yield node
        queue.dequeue()

def layerorder(self):
    return (node.data for node in self._layerorder())